In [1]:
import os 
import sys
import numpy as np 
import pandas as pd 
import scipy 
import matplotlib
import mne 
import EntropyHub as EH

from mne_features.bivariate import compute_phase_lock_val, compute_max_cross_corr

os.chdir('/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Scripts/Human/')
%run preprocess_human.py

/home/melissa/miniconda3/envs/env2023/lib/python3.8/site-packages/mne_features/utils.py:21: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit()


In [2]:
os.chdir('/home/melissa/RESULTS/XGBoost/Human_SYNGAP1/Cross_Corr/')
test = np.load('P1 N1_cross_corr.npy')
test_error = np.load('P1 N1_cross_corr.npy')
test_error.shape

(527, 21)

In [34]:
os.chdir('/home/melissa/RESULTS/XGBoost/Human_SYNGAP1/Cross_Corr_Channels/')
test = np.load('P1 N1_C3_M2.npy')

In [35]:
print(test.shape)
test


(527,)


array([0.36480151, 0.37934685, 0.39629576, 0.40677125, 0.42199435,
       0.43840005, 0.43965622, 0.42946436, 0.42347356, 0.42320147,
       0.42738164, 0.44040106, 0.45340066, 0.45235596, 0.43934284,
       0.42298612, 0.40900558, 0.40722467, 0.42082727, 0.4340508 ,
       0.43636819, 0.43616906, 0.43847764, 0.44051275, 0.44964731,
       0.47337708, 0.44971646, 0.42450268, 0.42701408, 0.44999241,
       0.47161832, 0.47754896, 0.46962906, 0.463831  , 0.46648099,
       0.49735199, 0.51325839, 0.5119687 , 0.49904172, 0.49157363,
       0.48809539, 0.45763674, 0.44574432, 0.44847436, 0.45696635,
       0.45021244, 0.42021184, 0.38891752, 0.40873138, 0.42168334,
       0.41879347, 0.42977395, 0.45471653, 0.48174607, 0.50032547,
       0.51028338, 0.51620035, 0.51634208, 0.50886837, 0.50109899,
       0.4980053 , 0.49403342, 0.48993899, 0.49826288, 0.51587086,
       0.51971547, 0.50479297, 0.50588683, 0.5132458 , 0.50309937,
       0.49324932, 0.50124908, 0.51133409, 0.50107999, 0.47651

In [16]:
os.chdir('/home/melissa/RESULTS/FINAL_MODEL/Human/Connectivity_MNE/xgb_dataframes/')
test_mne = pd.read_csv('P1 N1_all_conn_measures.csv')
len(test_mne['Idx'].to_list())

525

In [3]:
human_data_folder = '/home/melissa/PREPROCESSING/SYNGAP1/SYNGAP1_Human_Data'
results_path = '/home/melissa/RESULTS/FINAL_MODEL/Human/Cross_Corr/'
noise_directory = '/home/melissa/PREPROCESSING/SYNGAP1/human_npy/harmonic_idx/'
mne_directory = '/home/melissa/RESULTS/FINAL_MODEL/Human/Connectivity_MNE/xgb_dataframes/'

patient_list  =  [ 'P24 N1','P27 N1','P28 N1',
                  'P28 N2', 'P29 N2', 'P30 N1']
                 #'P6 N1', ['P1 N1', 'P2 N1', 'P2 N2', 'P3 N1', 'P3 N2', 'P4 N1', 'P4 N2', 'P5 N1',
                 #'P6 N2', 'P7 N1', 'P7 N2','P8 N1','P10 N1', 'P11 N1', 'P15 N1',
                 #'P16 N1', 'P17 N1', 'P18 N1','P20 N1', 'P21 N1', 'P21 N2', 'P21 N3',
                 #'P22 N1','P23 N1', 'P23 N2', 'P23 N3',


In [37]:
for patient in patient_list:
    print(patient)
    file_name = patient + '_(1).edf'
    filtered_data = load_filtered_data(file_path = human_data_folder, file_name = file_name)
    number_epochs, epochs = split_into_epochs(filtered_data, sampling_rate = 256, num_seconds = 30)
    epoch_array = np.array(epochs)
    
    #load mne features
    patient_mne = pd.read_csv(mne_directory + patient + '_all_conn_measures.csv')
    mne_indices = patient_mne['Idx'].to_list()
    
    cross_corr_ls_1 = []
    error_1 = []
    
    for idx in mne_indices:
        try:
            reshape_data = np.moveaxis(np.moveaxis((epoch_array), 1, 0), 2,1)
            one_epoch_1 = compute_max_cross_corr(sfreq = 256, data = reshape_data[:, idx]) 
            cross_corr_ls_1.append(one_epoch_1)
        except:
            print(str(patient) + ' error for index ' + str(idx))
            error_1.append(idx)

    cross_corr_1 = np.array(cross_corr_ls_1)
    error_array_1 = np.array(error_1)
    os.chdir(results_path)
    np.save(patient + '_cross_corr.npy', cross_corr_1)
    np.save(patient + '_error.npy', error_array_1)
    print('cross correlation values for ' + patient + ' saved')

P24 N1
Extracting EDF parameters from /home/melissa/PREPROCESSING/SYNGAP1/SYNGAP1_Human_Data/P24 N1_(1).edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 4389887  =      0.000 ... 17147.996 secs...


/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Scripts/Human/preprocess_human.py:14: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw_edf = mne.io.read_raw_edf(str(file_name),


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 2817 samples (11.004 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.6s finished


cross correlation values for P24 N1 saved
P27 N1
Extracting EDF parameters from /home/melissa/PREPROCESSING/SYNGAP1/SYNGAP1_Human_Data/P27 N1_(1).edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3652607  =      0.000 ... 14267.996 secs...


...

cross correlation values for P29 N2 saved
P30 N1
Extracting EDF parameters from /home/melissa/PREPROCESSING/SYNGAP1/SYNGAP1_Human_Data/P30 N1_(1).edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2982399  =      0.000 ... 11649.996 secs...


/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Scripts/Human/preprocess_human.py:14: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw_edf = mne.io.read_raw_edf(str(file_name),


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 2817 samples (11.004 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.4s finished


cross correlation values for P30 N1 saved


## Entropy 

In [3]:
os.chdir('/home/melissa/RESULTS/FINAL_MODEL/Human/Complexity/DispEn/')
test_en = np.load('P1 N1_01_dispen.npy')

In [9]:
human_data_folder = '/home/melissa/PREPROCESSING/SYNGAP1/SYNGAP1_Human_Data'
noise_directory = '/home/melissa/PREPROCESSING/SYNGAP1/human_npy/harmonic_idx/'
results_path = '/home/melissa/RESULTS/FINAL_MODEL/Human/Complexity/DispEn_DF/'

patient_list  =  [ 'P23 N3', 'P24 N1','P27 N1','P28 N1',
                  'P28 N2', 'P29 N2', 'P30 N1', 'P1 N1', 'P2 N1', 'P2 N2',
                  'P3 N1', 'P3 N2', 'P4 N1', 'P4 N2', 'P5 N1']
                  #'P6 N1','P6 N2', 'P7 N1', 'P7 N2','P8 N1','P10 N1', 'P11 N1', 'P15 N1',
                  #'P16 N1', 'P17 N1', 'P18 N1','P20 N1', 'P21 N1', 'P21 N2', 'P21 N3',
                  #'P22 N1','P23 N1','P23 N2',

# Define a function to calculate DispEn for a channel
def calculate_dispen(data_input, patient_id, channel_list, clean_indices, results_path):
    
    all_indices = []
    
    for clean_idx in clean_indices:
        all_channels = []
        for chan_idx, channel in enumerate(channel_list):
            Dispx, RDE = EH.DispEn(Sig = np.array(data_input[clean_idx][chan_idx, :]), m=3, tau=2, c=4, Typex='ncdf')
            dispen_dict = {str(channel) + '_dispen': [Dispx], str(channel) + '_RDE': [RDE]}
            dispen_df = pd.DataFrame(data = dispen_dict)
            all_channels.append(dispen_df)
        dispen_idx = pd.concat(all_channels, axis = 1)
        all_indices.append(dispen_idx)
    dispen_all_epochs = pd.concat(all_indices, axis = 0)
    
    dispen_all_epochs.to_csv(results_path + str(patient_id) + '_dispen.csv')
    

In [10]:
for patient in patient_list:
    print(patient)
    file_name = patient + '_(1).edf'
    filtered_data = load_filtered_data(file_path = human_data_folder, file_name = file_name)
    number_epochs, epochs = split_into_epochs(filtered_data, sampling_rate = 256, num_seconds = 30)
    epoch_array = np.array(epochs)
    
    #load mne features
    patient_mne = pd.read_csv(mne_directory + patient + '_all_conn_measures.csv')
    mne_indices = patient_mne['Idx'].to_list()
    
    # Calculate DispEn for each channel
    channels = ['E1', 'E2', 'F3', 'C3', 'O1', 'M2']
 
    calculate_dispen(data_input = epochs, patient_id = patient,
                         channel_list = channels,
                         clean_indices = mne_indices,
                         results_path = results_path)
    
    print('All channels calculated')


P23 N3
Extracting EDF parameters from /home/melissa/PREPROCESSING/SYNGAP1/SYNGAP1_Human_Data/P23 N3_(1).edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 4792063  =      0.000 ... 18718.996 secs...


/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Scripts/Human/preprocess_human.py:14: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw_edf = mne.io.read_raw_edf(str(file_name),


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 2817 samples (11.004 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.7s finished


All channels calculated
P24 N1
Extracting EDF parameters from /home/melissa/PREPROCESSING/SYNGAP1/SYNGAP1_Human_Data/P24 N1_(1).edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 4389887  =      0.000 ... 17147.996 secs...


...

All channels calculated
P4 N2
Extracting EDF parameters from /home/melissa/PREPROCESSING/SYNGAP1/SYNGAP1_Human_Data/P4 N2_(1).edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 10629119  =      0.000 ... 41519.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 2817 samples (11.004 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.5s finished


All channels calculated
P5 N1
Extracting EDF parameters from /home/melissa/PREPROCESSING/SYNGAP1/SYNGAP1_Human_Data/P5 N1_(1).edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 12901887  =      0.000 ... 50397.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 2817 samples (11.004 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.8s finished


All channels calculated


In [52]:
epochs[0][1, :].shape

(7680,)